In [ ]:
# default_exp dataset.image_segmentation

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import numpy as np
import sys
import argparse
import logging
from os.path import join, isfile, dirname, basename, splitext
from functools import partial
from datetime import datetime
from logging.handlers import MemoryHandler
from mlcore.core import Type, infer_type
from mlcore.dataset.core import input_feedback
from mlcore.dataset.image_object_detection import ImageObjectDetectionDataSet
from mlcore.image.pillow_tools import assign_exif_orientation, get_image_size, write_mask
from mlcore.io.core import create_folder
from mlcore.annotation.core import RegionShape, convert_region, region_bounding_box

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

CATEGORY_LABEL_KEY = 'category'
DEFAULT_CATEGORIES_FILE = 'categories.txt'
DEFAULT_CLASSIFICATION_ANNOTATIONS_FILE = 'annotations.csv'
DEFAULT_SEGMENTATION_ANNOTATIONS_FILE = 'via_region_data.json'
DEFAULT_SPLIT = 0.2
DATA_SET_FOLDER = 'datasets'
SEMANTIC_MASK_FOLDER = 'semantic_masks'
TRAIN_VAL_FOLDER = 'trainval'
TRAIN_FOLDER = 'train'
VAL_FOLDER = 'val'
TEST_FOLDER = 'test'
NOT_CATEGORIZED = '[NOT_CATEGORIZED]'

In [ ]:
# export

logger = logging.getLogger(__name__)

# Data-Set

> Data-Set Notes.

Creating a data-set for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The data-set is created based on an image-set.

## Image-Set

Image-sets are collected images to build a data-set from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[image_set_type]*/*[image_set_name]*

Inside the `[image_set_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the image-set contains

## Data-Set Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[data_set_type]*/*[data_set_name]*
where `[data_set_type]` is the same as the corresponding `[image_set_type]` and `[data_set_name]` is the same as the corresponding `[image_set_name]`.

Inside the `[data_set_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the data-set contains

## Create a segmentation data-set

Segmentation data-set can be created from a segmentation image-set.
All images are validated against the annotations, if they contain at least one annotation and that the annotation category belongs to one of the given categories. The annotations have to be in [VIA v1](http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.5.html) json format. Rectangle annotations are converted into polygon annotations for unique segment generation.

In [ ]:
# export


class ImageSegmentationDataSet(ImageObjectDetectionDataSet):
    def __init__(self, name, base_path, image_set_path, categories_path, data_set_type, annotations_path=None):
        super().__init__(name, base_path, image_set_path, categories_path, data_set_type, annotations_path)
        self.semantic_mask_folder = None

    def create_folders(self):
        """
        Creates the data-set folder structure, if not exist
        """
        super().create_folders()

        # create semantic mask file folder and remove previous data if exist
        self.semantic_mask_folder = create_folder(join(self.folder, SEMANTIC_MASK_FOLDER), clear=True)
        self.logger.info("Created folder {}".format(self.semantic_mask_folder))

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the data-set, generate the annotations for train and val images
        and generate the semantic masks.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        annotations_train, annotations_val = super().copy(train_file_keys, val_file_keys, test_file_names)
        # save semantic masks
        self._save_semantic_masks(train_file_keys + val_file_keys)
        return annotations_train, annotations_val

    def convert_annotations(self):
        """
        Converts segmentation regions from rectangle to polygon, if exist
        """

        # only the trainval images have annotation, not the test images
        steps = [
            {
                'name': 'position',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    't': 'Trim',  # transform the annotation
                    'T': 'Trim All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, size: p_min < 0 or p_max >= size,
                'message': '{} -> {} : {}Exceeds image {}. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: max(min(p, size - 1), 0),
            },
            {
                'name': 'size',
                'choices': {
                    's': 'Skip',  # just delete the annotation
                    'S': 'Skip All',
                    'k': 'Keep',  # transform the annotation (in this case do nothing)
                    'K': 'Keep All',
                },
                'choice': None,
                'condition': lambda p_min, p_max, _: p_max - p_min <= 1,
                'message': '{} -> {} : {}Shape {} is <= 1 pixel. \n Points \n x: {} \n y: {}',
                'transform': lambda p, size=0: p,
            }
        ]

        self.logger.info('Start convert image annotations from {}'.format(self.annotations_path))

        for annotation in self.annotations.values():
            # skip file, if regions are empty or file do not exist
            if not (annotation.regions and isfile(annotation.file_path)):
                continue

            image, _, __ = assign_exif_orientation(annotation.file_path)
            image_width, image_height = image.size

            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                # convert from rect to polygon if needed
                convert_region(region, RegionShape.POLYGON)

                for step in steps:
                    # validate the shape size
                    (x_min, x_max), (y_min, y_max) = region_bounding_box(region)

                    width_condition = step['condition'](x_min, x_max, image_width)
                    height_condition = step['condition'](y_min, y_max, image_height)
                    if width_condition or height_condition:
                        size_message = ['width'] if width_condition else []
                        size_message.extend(['height'] if height_condition else [])
                        message = step['message'].format(annotation.file_name, index, ' ', ' and '.join(size_message),
                                                         region.points_x, region.points_y)

                        step['choice'] = input_feedback(message, step['choice'], step['choices'])

                        choice_op = step['choice'].lower()
                        # if skip the shapes
                        if choice_op == 's':
                            delete_regions[index] = True
                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             region.points_x, region.points_y)
                            self.logger.info(message)

                            break
                        else:
                            annotation.points_x = list(map(partial(step['transform'], size=image_width),
                                                           annotation.points_x))
                            annotation.points_y = list(map(partial(step['transform'], size=image_height),
                                                           annotation.points_y))

                            message = step['message'].format(annotation.file_name, index,
                                                             '{} '.format(step['choices'][choice_op]),
                                                             ' and '.join(size_message),
                                                             region.points_x, region.points_y)
                            self.logger.info(message)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

        print('Finished convert image annotations from {}'.format(self.annotations_path))

    def _save_semantic_masks(self, keys):
        """
        Create semantic segmentation mask png files out of the annotations.
        The mask file name is the same as the image file name but is stored in png format.
        `keys`: The annotation keys to create the segmentation masks for
        """
        from skimage import draw

        num_masks = len(keys)
        self.logger.info('Start create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

        # only the trainval images have annotation, not the test images
        for index, key in enumerate(keys):
            annotation = self.annotations[key]

            if not annotation.annotations:
                continue

            image, image_width, image_height = get_image_size(annotation.file_path)

            # Convert polygons to a bitmap mask of shape
            # [height, width]
            mask = np.zeros((image_height, image_width), dtype=np.uint8)

            # sort the regions by category priority for handling pixels which are assigned to more than one category
            # the category with higher index overpaint the category with lower index
            for region in sorted(annotation.regions(), key=lambda r: self.categories.index(r.labels[0])):
                class_id = self.categories.index(region.labels[0]) + 1

                # Get indexes of pixels inside the polygon and set them to 1
                rr, cc = draw.polygon(region.points_y, region.points_x)
                mask[rr, cc] = class_id

            # save the semantic mask
            mask_path = join(self.semantic_mask_folder, splitext(annotation.file_name)[0] + '.png')
            write_mask(mask, mask_path)

            self.logger.info('{} / {} - Created segmentation mask {}'.format(index + 1, num_masks, mask_path))

        self.logger.info('Finish create {} segmentation masks in {}'.format(num_masks, self.semantic_mask_folder))

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.
    """
    log_memory_handler = MemoryHandler(1, flushLevel=logging_level)
    log_memory_handler.setLevel(logging_level)

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging_level)

    logger.addHandler(log_memory_handler)
    logger.addHandler(stdout_handler)

    logger.setLevel(logging_level)

    return log_memory_handler

## Build a data-set

To build a data-set from an image-set. Handles currently classification and segmentation image-sets taken from the image-set-type, which is the parent folder, the image-set folder is located in. 

In [ ]:
# export


def build_data_set(category_file_path, output, annotation_file_path=None, split=DEFAULT_SPLIT, seed=None, sample=0,
                   data_set_type=None, create_tfrecord=False, join_overlapping_regions=False,
                   annotation_area_threshold=None, data_set_name=None):
    """
    Build the data-set for training, Validation and test
    `category_file_path`: the filename of the categories file
    `output`: the dataset base folder to build the dataset in
    `annotation_file_path`: the file path to the annotation file
    `split`: the size of the validation set as percentage
    `seed`: random seed to reproduce splits
    `sample`: the size of the sample set as percentage
    `data_set_type`: the type of the data-set, if not set infer from the category file path
    `create_tfrecord`: Also create .tfrecord files.
    `join_overlapping_regions`: Whether overlapping regions of same category should be joined.
    `annotation_area_threshold`: Keep only annotations with minimum size (width or height) related to image size
    `data_set_name`: the name of the data-set, if not set infer from the category file path
    """
    log_memory_handler = configure_logging()

    # try to infer the data-set type if not explicitly set
    if data_set_type is None:
        try:
            data_set_type = infer_type(category_file_path)
        except ValueError as e:
            logger.error(e)
            return

    path = dirname(category_file_path)

    # try to infer the data-set name if not explicitly set
    if data_set_name is None:
        data_set_name = basename(path)

    logger.info('Build parameters:')
    logger.info(' '.join(sys.argv[1:]))
    logger.info('Build configuration:')
    logger.info('category_file_path: {}'.format(category_file_path))
    logger.info('annotation_file_path: {}'.format(annotation_file_path))
    logger.info('split: {}'.format(split))
    logger.info('seed: {}'.format(seed))
    logger.info('sample: {}'.format(sample))
    logger.info('type: {}'.format(data_set_type))
    logger.info('output: {}'.format(output))
    logger.info('join_overlapping_regions: {}'.format(join_overlapping_regions))
    logger.info('annotation_area_threshold: {}'.format(annotation_area_threshold))
    logger.info('name: {}'.format(data_set_name))

    data_set = None
    logger.info('Start build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

    if data_set_type == Type.IMAGE_CLASSIFICATION:
        data_set = ClassificationDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                         annotation_file_path)
    elif data_set_type == Type.IMAGE_SEGMENTATION:
        data_set = SegmentationDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                       annotation_file_path)
    elif data_set_type == Type.IMAGE_OBJECT_DETECTION:
        data_set = ObjectDetectionDataSet(data_set_name, output, path, category_file_path, data_set_type,
                                          annotation_file_path, create_tfrecord, join_overlapping_regions,
                                          annotation_area_threshold)

    if data_set:
        # create the data set folders
        logger.info("Start create the data-set folders at {}".format(data_set.base_path))
        data_set.create_folders()
        logger.info("Finished create the data-set folders at {}".format(data_set.base_path))

        # create the build log file
        log_file_name = datetime.now().strftime("build_%Y.%m.%d-%H.%M.%S.log")
        file_handler = logging.FileHandler(join(data_set.folder, log_file_name), encoding="utf-8")
        log_memory_handler.setTarget(file_handler)

        # build the data set
        data_set.build(split, seed, sample)

    logger.info('Finished build {} data-set {} at {}'.format(data_set_type, data_set_name, output))

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.dataset [parameters]`

The following parameters are supported:
- `[categories]`: The path to the categories file. (e.g.: *categories.txt*)
- `--annotation`: The path to the image-set annotation file, the data-set is build from. (e.g.: *imagesets/classification/car_damage/annotations.csv* for classification, *imagesets/segmentation/car_damage/via_region_data.json* for segmentation)
- `--split`: The percentage of the data which belongs to validation set, default to *0.2* (=20%)
- `--seed`: A random seed to reproduce splits, default to None
- `--category-label-key`: The key, the category name can be found in the annotation file, default to *category*.
- `--sample`: The percentage of the data which will be copied as a sample set with in a separate folder with "_sample" suffix. If not set, no sample data-set will be created.
- `--type`: The type of the data-set, if not explicitly set try to infer from categories file path.
- `--tfrecord`: Also create .tfrecord files.
- `--join-overlapping-regions`: Whether overlapping regions of same category should be joined.
- `--annotation-area-thresh`: Keep only annotations with minimum size (width or height) related to image size.
- `--output`: The path of the dataset folder, default to *../datasets*.
- `--name`: The name of the data-set, if not explicitly set try to infer from categories file path.

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    parser = argparse.ArgumentParser()
    parser.add_argument("categories",
                        help="The path to the image-set categories file.")
    parser.add_argument("--annotation",
                        help="The path to the image-set annotation file, the data-set is build from.",
                        default=None)
    parser.add_argument("--split",
                        help="Percentage of the data which belongs to validation set.",
                        type=float,
                        default=0.2)
    parser.add_argument("--seed",
                        help="A random seed to reproduce splits.",
                        type=int,
                        default=None)
    parser.add_argument("--category-label-key",
                        help="The key of the category name.",
                        default=CATEGORY_LABEL_KEY)
    parser.add_argument("--sample",
                        help="Percentage of the data which will be copied as a sample set.",
                        type=float,
                        default=0)
    parser.add_argument("--type",
                        help="The type of the data-set, if not explicitly set try to infer from categories file path.",
                        choices=list(Type),
                        type=Type,
                        default=None)
    parser.add_argument("--tfrecord",
                        help="Also create .tfrecord files.",
                        action="store_true")
    parser.add_argument("--join-overlapping-regions",
                        help="Whether overlapping regions of same category should be joined.",
                        action="store_true")
    parser.add_argument("--annotation-area-thresh",
                        help="Keep only annotations with minimum size (width or height) related to image size.",
                        type=float,
                        default=None)
    parser.add_argument("--output",
                        help="The path of the data-set folder.",
                        default=DATA_SET_FOLDER)
    parser.add_argument("--name",
                        help="The name of the data-set, if not explicitly set try to infer from categories file path.",
                        default=None)
    args = parser.parse_args()

    CATEGORY_LABEL_KEY = args.category_label_key

    build_data_set(args.categories, args.output, args.annotation, args.split, args.seed, args.sample, args.type,
                   args.tfrecord, args.join_overlapping_regions, args.annotation_area_thresh, args.name)
